In [2]:
%cd ../

/home/hoanghu/projects/Food-Waste-Optimization


In [20]:
from pathlib import Path

import numpy as np
from transformers import pipeline

In [5]:
path = "dish_names.txt.npy"

dish_names = np.load(path, allow_pickle=True)

In [7]:
pipeline = pipeline('feature-extraction', model='Finnish-NLP/gpt2-medium-finnish')


/home/hoanghu/miniforge3/envs/fw/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
data = pipeline(dish_names.tolist())
vec = np.array(data, dtype=np.float32)

vec.shape

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (271, 1) + inhomogeneous part.

In [18]:
vectors = {}

for dish_name, vec in zip (dish_names, data):
    vectors[dish_name] = np.float32(vec).squeeze().sum(axis=0)

In [23]:
path_embds = Path("experiments_hoangle/hypotheses") / "names_embd.npy"

np.save(path_embds, vectors)